In [4]:
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")

import pickle 

from sklearn.utils import shuffle

In [5]:
# Read in datasets of combined flight and weather data
# All2017Flight_Weather.csv is the file name from SettingUpCSVTrainingYearly.ipynb
df18 = pd.read_csv('All2018Round2_InDelay.csv', index_col=None)
df18.head(3).append(df18.tail(3))
df18 = df18.loc[:, ~df18.columns.str.contains('^Unnamed')]

In [6]:
df18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1890179 entries, 0 to 1890178
Data columns (total 88 columns):
CRS_DEP_TIME              int64
CRS_ARR_TIME              int64
ARR_DELAY                 float64
AIR_TIME                  float64
DISTANCE                  float64
T_DAILY_MEAN_Midwest      float64
P_DAILY_CALC_Midwest      float64
T_DAILY_MEAN_Mountain     float64
P_DAILY_CALC_Mountain     float64
T_DAILY_MEAN_Northeast    float64
P_DAILY_CALC_Northeast    float64
T_DAILY_MEAN_PNW          float64
P_DAILY_CALC_PNW          float64
T_DAILY_MEAN_Southeast    float64
P_DAILY_CALC_Southeast    float64
T_DAILY_MEAN_Southwest    float64
P_DAILY_CALC_Southwest    float64
T_DAILY_MEAN_Texas        float64
P_DAILY_CALC_Texas        float64
DAY                       int64
MONTH                     int64
HOLIDAY                   int64
FLIGHT_STATUS             int64
INCOMING_Delay            int64
OP_CARRIER_AA             int64
OP_CARRIER_DL             int64
OP_CARRIER_UA        

In [7]:
df18.head()

,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,T_DAILY_MEAN_Midwest,P_DAILY_CALC_Midwest,T_DAILY_MEAN_Mountain,P_DAILY_CALC_Mountain,T_DAILY_MEAN_Northeast,P_DAILY_CALC_Northeast,T_DAILY_MEAN_PNW,P_DAILY_CALC_PNW,T_DAILY_MEAN_Southeast,P_DAILY_CALC_Southeast,T_DAILY_MEAN_Southwest,P_DAILY_CALC_Southwest,T_DAILY_MEAN_Texas,P_DAILY_CALC_Texas,DAY,MONTH,HOLIDAY,FLIGHT_STATUS,INCOMING_Delay,OP_CARRIER_AA,OP_CARRIER_DL,OP_CARRIER_UA,OP_CARRIER_WN,ORIGIN_ATL,ORIGIN_AUS,ORIGIN_BNA,ORIGIN_BOS,ORIGIN_BWI,ORIGIN_CLT,ORIGIN_DCA,ORIGIN_DEN,ORIGIN_DFW,ORIGIN_DTW,ORIGIN_EWR,ORIGIN_FLL,ORIGIN_IAD,ORIGIN_IAH,ORIGIN_JFK,ORIGIN_LAS,ORIGIN_LAX,ORIGIN_LGA,ORIGIN_MCO,ORIGIN_MDW,ORIGIN_MIA,ORIGIN_MSP,ORIGIN_ORD,ORIGIN_PHL,ORIGIN_PHX,ORIGIN_SAN,ORIGIN_SEA,ORIGIN_SFO,ORIGIN_SLC,ORIGIN_TPA,DEST_ATL,DEST_AUS,DEST_BNA,DEST_BOS,DEST_BWI,DEST_CLT,DEST_DCA,DEST_DEN,DEST_DFW,DEST_DTW,DEST_EWR,DEST_FLL,DEST_IAD,DEST_IAH,DEST_JFK,DEST_LAS,DEST_LAX,DEST_LGA,DEST_MCO,DEST_MDW,DEST_MIA,DEST_MSP,DEST_ORD,DEST_PHL,DEST_PHX,DEST_SAN,DEST_SEA,DEST_SFO,DEST_SLC,DEST_TPA
0,1517,1745,-23.0,225.0,1605.0,-21.5,0.0,-3.9,0.0,-17.3,0.0,2.5,0.0,-3.3,0.0,11.2,0.0,-4.7,0.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1115,1254,-24.0,65.0,414.0,-21.5,0.0,-3.9,0.0,-17.3,0.0,2.5,0.0,-3.3,0.0,11.2,0.0,-4.7,0.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,750,916,-16.0,173.0,1222.0,-21.5,0.0,-3.9,0.0,-17.3,0.0,2.5,0.0,-3.3,0.0,11.2,0.0,-4.7,0.0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1044,1318,-3.0,124.0,746.0,-21.5,0.0,-3.9,0.0,-17.3,0.0,2.5,0.0,-3.3,0.0,11.2,0.0,-4.7,0.0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,653,930,55.0,302.0,2218.0,-21.5,0.0,-3.9,0.0,-17.3,0.0,2.5,0.0,-3.3,0.0,11.2,0.0,-4.7,0.0,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# Shuffle dataset to randomly select validation and testing datasets
df18_shuf = shuffle(df18)
df18_shuf.head()

,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,T_DAILY_MEAN_Midwest,P_DAILY_CALC_Midwest,T_DAILY_MEAN_Mountain,P_DAILY_CALC_Mountain,T_DAILY_MEAN_Northeast,P_DAILY_CALC_Northeast,T_DAILY_MEAN_PNW,P_DAILY_CALC_PNW,T_DAILY_MEAN_Southeast,P_DAILY_CALC_Southeast,T_DAILY_MEAN_Southwest,P_DAILY_CALC_Southwest,T_DAILY_MEAN_Texas,P_DAILY_CALC_Texas,DAY,MONTH,HOLIDAY,FLIGHT_STATUS,INCOMING_Delay,OP_CARRIER_AA,OP_CARRIER_DL,OP_CARRIER_UA,OP_CARRIER_WN,ORIGIN_ATL,ORIGIN_AUS,ORIGIN_BNA,ORIGIN_BOS,ORIGIN_BWI,ORIGIN_CLT,ORIGIN_DCA,ORIGIN_DEN,ORIGIN_DFW,ORIGIN_DTW,ORIGIN_EWR,ORIGIN_FLL,ORIGIN_IAD,ORIGIN_IAH,ORIGIN_JFK,ORIGIN_LAS,ORIGIN_LAX,ORIGIN_LGA,ORIGIN_MCO,ORIGIN_MDW,ORIGIN_MIA,ORIGIN_MSP,ORIGIN_ORD,ORIGIN_PHL,ORIGIN_PHX,ORIGIN_SAN,ORIGIN_SEA,ORIGIN_SFO,ORIGIN_SLC,ORIGIN_TPA,DEST_ATL,DEST_AUS,DEST_BNA,DEST_BOS,DEST_BWI,DEST_CLT,DEST_DCA,DEST_DEN,DEST_DFW,DEST_DTW,DEST_EWR,DEST_FLL,DEST_IAD,DEST_IAH,DEST_JFK,DEST_LAS,DEST_LAX,DEST_LGA,DEST_MCO,DEST_MDW,DEST_MIA,DEST_MSP,DEST_ORD,DEST_PHL,DEST_PHX,DEST_SAN,DEST_SEA,DEST_SFO,DEST_SLC,DEST_TPA
1558158,1444,1818,-7.0,120.0,967.0,6.7,0.0,3.5,0.0,2.2,0.0,10.7,38.9,8.8,30.4,15.7,0.0,18.3,0.0,26,10,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
637643,1542,1816,-26.0,116.0,921.0,20.7,0.0,6.9,0.0,14.6,0.0,15.7,7.4,19.5,0.0,14.7,0.0,24.6,0.0,8,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2183,1045,1340,6.0,152.0,992.0,-21.5,0.0,-3.9,0.0,-17.3,0.0,2.5,0.0,-3.3,0.0,11.2,0.0,-4.7,0.0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1348215,1000,1123,-7.0,44.0,184.0,24.0,6.4,15.3,0.0,21.1,4.9,11.4,0.0,26.0,0.0,16.9,0.0,26.0,0.0,17,9,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4291,1505,1711,39.0,122.0,666.0,-21.5,0.0,-3.9,0.0,-17.3,0.0,2.5,0.0,-3.3,0.0,11.2,0.0,-4.7,0.0,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [9]:
df18_shuf.reset_index(inplace=True, drop=True)
df18_shuf.head()

,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,T_DAILY_MEAN_Midwest,P_DAILY_CALC_Midwest,T_DAILY_MEAN_Mountain,P_DAILY_CALC_Mountain,T_DAILY_MEAN_Northeast,P_DAILY_CALC_Northeast,T_DAILY_MEAN_PNW,P_DAILY_CALC_PNW,T_DAILY_MEAN_Southeast,P_DAILY_CALC_Southeast,T_DAILY_MEAN_Southwest,P_DAILY_CALC_Southwest,T_DAILY_MEAN_Texas,P_DAILY_CALC_Texas,DAY,MONTH,HOLIDAY,FLIGHT_STATUS,INCOMING_Delay,OP_CARRIER_AA,OP_CARRIER_DL,OP_CARRIER_UA,OP_CARRIER_WN,ORIGIN_ATL,ORIGIN_AUS,ORIGIN_BNA,ORIGIN_BOS,ORIGIN_BWI,ORIGIN_CLT,ORIGIN_DCA,ORIGIN_DEN,ORIGIN_DFW,ORIGIN_DTW,ORIGIN_EWR,ORIGIN_FLL,ORIGIN_IAD,ORIGIN_IAH,ORIGIN_JFK,ORIGIN_LAS,ORIGIN_LAX,ORIGIN_LGA,ORIGIN_MCO,ORIGIN_MDW,ORIGIN_MIA,ORIGIN_MSP,ORIGIN_ORD,ORIGIN_PHL,ORIGIN_PHX,ORIGIN_SAN,ORIGIN_SEA,ORIGIN_SFO,ORIGIN_SLC,ORIGIN_TPA,DEST_ATL,DEST_AUS,DEST_BNA,DEST_BOS,DEST_BWI,DEST_CLT,DEST_DCA,DEST_DEN,DEST_DFW,DEST_DTW,DEST_EWR,DEST_FLL,DEST_IAD,DEST_IAH,DEST_JFK,DEST_LAS,DEST_LAX,DEST_LGA,DEST_MCO,DEST_MDW,DEST_MIA,DEST_MSP,DEST_ORD,DEST_PHL,DEST_PHX,DEST_SAN,DEST_SEA,DEST_SFO,DEST_SLC,DEST_TPA
0,1444,1818,-7.0,120.0,967.0,6.7,0.0,3.5,0.0,2.2,0.0,10.7,38.9,8.8,30.4,15.7,0.0,18.3,0.0,26,10,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1542,1816,-26.0,116.0,921.0,20.7,0.0,6.9,0.0,14.6,0.0,15.7,7.4,19.5,0.0,14.7,0.0,24.6,0.0,8,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1045,1340,6.0,152.0,992.0,-21.5,0.0,-3.9,0.0,-17.3,0.0,2.5,0.0,-3.3,0.0,11.2,0.0,-4.7,0.0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1000,1123,-7.0,44.0,184.0,24.0,6.4,15.3,0.0,21.1,4.9,11.4,0.0,26.0,0.0,16.9,0.0,26.0,0.0,17,9,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,1505,1711,39.0,122.0,666.0,-21.5,0.0,-3.9,0.0,-17.3,0.0,2.5,0.0,-3.3,0.0,11.2,0.0,-4.7,0.0,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [10]:
# Separate testing and validation
# This method includs a small amoutn of data leakage in the event that one optimizes on validation data
# Ex: If 8/1/2018 is in validation and 8/2/2018 is in testing, then massive thunderstorms in Chicago and NY on 8/1/2018, the
# are likely to cascade into 8/2/2018.
# However, I experiemented with methods of selecting dates that had more data leakage and they had <0.5% effects on model accuracy
df18_test = df18_shuf.iloc[:945089,:]
df18_val = df18_shuf.iloc[945089:,:]

In [11]:
df18_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945089 entries, 0 to 945088
Data columns (total 88 columns):
CRS_DEP_TIME              945089 non-null int64
CRS_ARR_TIME              945089 non-null int64
ARR_DELAY                 931702 non-null float64
AIR_TIME                  931702 non-null float64
DISTANCE                  945089 non-null float64
T_DAILY_MEAN_Midwest      945089 non-null float64
P_DAILY_CALC_Midwest      945089 non-null float64
T_DAILY_MEAN_Mountain     945089 non-null float64
P_DAILY_CALC_Mountain     945089 non-null float64
T_DAILY_MEAN_Northeast    945089 non-null float64
P_DAILY_CALC_Northeast    945089 non-null float64
T_DAILY_MEAN_PNW          945089 non-null float64
P_DAILY_CALC_PNW          945089 non-null float64
T_DAILY_MEAN_Southeast    945089 non-null float64
P_DAILY_CALC_Southeast    945089 non-null float64
T_DAILY_MEAN_Southwest    945089 non-null float64
P_DAILY_CALC_Southwest    945089 non-null float64
T_DAILY_MEAN_Texas        945089 non-null f

In [12]:
df18_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945090 entries, 945089 to 1890178
Data columns (total 88 columns):
CRS_DEP_TIME              945090 non-null int64
CRS_ARR_TIME              945090 non-null int64
ARR_DELAY                 931425 non-null float64
AIR_TIME                  931425 non-null float64
DISTANCE                  945090 non-null float64
T_DAILY_MEAN_Midwest      945090 non-null float64
P_DAILY_CALC_Midwest      945090 non-null float64
T_DAILY_MEAN_Mountain     945090 non-null float64
P_DAILY_CALC_Mountain     945090 non-null float64
T_DAILY_MEAN_Northeast    945090 non-null float64
P_DAILY_CALC_Northeast    945090 non-null float64
T_DAILY_MEAN_PNW          945090 non-null float64
P_DAILY_CALC_PNW          945090 non-null float64
T_DAILY_MEAN_Southeast    945090 non-null float64
P_DAILY_CALC_Southeast    945090 non-null float64
T_DAILY_MEAN_Southwest    945090 non-null float64
P_DAILY_CALC_Southwest    945090 non-null float64
T_DAILY_MEAN_Texas        945090 non-

In [13]:
df18_test = df18_test.dropna(how='any',axis=0) # Remove Null NaN values
df18_val = df18_val.dropna(how='any',axis=0) # Remove Null NaN values

In [14]:
# df0916.to_csv('All0916Train.csv')
df18_test.to_pickle("All18Test_InDelay.pkl")
df18_val.to_pickle("All18Val_InDelay.pkl")

In [15]:
# df1.head()
df18_test.info()
df18_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 931702 entries, 0 to 945088
Data columns (total 88 columns):
CRS_DEP_TIME              931702 non-null int64
CRS_ARR_TIME              931702 non-null int64
ARR_DELAY                 931702 non-null float64
AIR_TIME                  931702 non-null float64
DISTANCE                  931702 non-null float64
T_DAILY_MEAN_Midwest      931702 non-null float64
P_DAILY_CALC_Midwest      931702 non-null float64
T_DAILY_MEAN_Mountain     931702 non-null float64
P_DAILY_CALC_Mountain     931702 non-null float64
T_DAILY_MEAN_Northeast    931702 non-null float64
P_DAILY_CALC_Northeast    931702 non-null float64
T_DAILY_MEAN_PNW          931702 non-null float64
P_DAILY_CALC_PNW          931702 non-null float64
T_DAILY_MEAN_Southeast    931702 non-null float64
P_DAILY_CALC_Southeast    931702 non-null float64
T_DAILY_MEAN_Southwest    931702 non-null float64
P_DAILY_CALC_Southwest    931702 non-null float64
T_DAILY_MEAN_Texas        931702 non-null f